In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, 'C:/Users/User/Desktop/Week-3/scripts')
df= pd.read_csv('C:/Users/User/Desktop/10/MachineLearningRating_v3/MachineLearningRating_v3.txt', delimiter='|', low_memory=False)
# Preview the data
df.head()


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


<font size ="17"><b>Selecting Matrics</b></font>

In [2]:
# Calculate profit margin
df['ProfitMargin'] = df['TotalPremium'] - df['TotalClaims']

# Verify the metrics
df[['TotalPremium', 'TotalClaims', 'ProfitMargin']].head()

,TotalPremium,TotalClaims,ProfitMargin
0,21.929825,0.0,21.929825
1,21.929825,0.0,21.929825
2,0.000000,0.0,0.000000
3,512.848070,0.0,512.848070
4,0.000000,0.0,0.000000


<font size="16"><b>Data Segmentation and Statistical Testing</b></font>

In [14]:
from scipy.stats import ttest_ind, chi2_contingency

# Drop missing values
df_clean = df.dropna(subset=['TotalClaims', 'PostalCode', 'Province', 'Gender'])

# Function to perform T-test
def perform_ttest(group_A, group_B, column):
    if len(group_A) == 0 or len(group_B) == 0:
        print("One of the groups is empty, cannot perform T-test.")
        return None
    t_stat, p_value = ttest_ind(group_A[column], group_B[column], equal_var=False)
    print(f'T-statistic: {t_stat}, P-value: {p_value}')
    return p_value

# Function to perform Chi-Square test
def perform_chi2_test(df, column_A, column_B):
    contingency_table = pd.crosstab(df[column_A], df[column_B])
    if contingency_table.size == 0:
        print("Contingency table is empty, cannot perform Chi2 test.")
        return None
    chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
    print(f'Chi2 Stat: {chi2_stat}, P-value: {p_val}')
    return p_val

# Select two different groups dynamically
def select_groups(df, column):
    unique_values = df[column].unique()
    if len(unique_values) < 2:
        print(f"Not enough unique values in {column} to create two groups.")
        return None, None
    return unique_values[0], unique_values[1] 

In [15]:
# 1. Risk Differences Across Provinces
def test_risk_by_province(df):
    print("Testing risk differences across provinces:")
    
    # Select two different provinces
    province_A, province_B = select_groups(df, 'Province')
    if province_A is None or province_B is None:
        return
    
    group_A = df[df['Province'] == province_A]
    group_B = df[df['Province'] == province_B]
    
    # Perform T-test on Total Claims
    p_value = perform_ttest(group_A, group_B, 'TotalClaims')
    
    if p_value and p_value < 0.05:
        print(f"Reject the null hypothesis: There is a significant difference in risk between {province_A} and {province_B}.")
    else:
        print(f"Fail to reject the null hypothesis: No significant difference in risk between {province_A} and {province_B}.")

In [16]:
# 2. Risk Differences Between Zip Codes
def test_risk_by_zipcode(df):
    print("Testing risk differences between zip codes:")
    
    # Select two different zip codes
    zip_A, zip_B = select_groups(df, 'PostalCode')
    if zip_A is None or zip_B is None:
        return
    
    group_A = df[df['PostalCode'] == zip_A]
    group_B = df[df['PostalCode'] == zip_B]
    
    # Perform T-test on Total Claims
    p_value = perform_ttest(group_A, group_B, 'TotalClaims')
    
    if p_value and p_value < 0.05:
        print(f"Reject the null hypothesis: There is a significant difference in risk between zip codes {zip_A} and {zip_B}.")
    else:
        print(f"Fail to reject the null hypothesis: No significant difference in risk between zip codes {zip_A} and {zip_B}.")

In [17]:
# 3. Profit Margin Differences Between Zip Codes
def test_profit_margin_by_zipcode(df):
    print("Testing profit margin differences between zip codes:")
    
    # Select two different zip codes
    zip_A, zip_B = select_groups(df, 'PostalCode')
    if zip_A is None or zip_B is None:
        return
    
    # Create a ProfitMargin column if it doesn't exist
    if 'ProfitMargin' not in df.columns:
        df['ProfitMargin'] = df['TotalPremium'] - df['TotalClaims']
    
    group_A = df[df['PostalCode'] == zip_A]
    group_B = df[df['PostalCode'] == zip_B]
    
    # Perform T-test on Profit Margin
    p_value = perform_ttest(group_A, group_B, 'ProfitMargin')
    
    if p_value and p_value < 0.05:
        print(f"Reject the null hypothesis: There is a significant difference in profit margin between zip codes {zip_A} and {zip_B}.")
    else:
        print(f"Fail to reject the null hypothesis: No significant difference in profit margin between zip codes {zip_A} and {zip_B}.")
